## Chloebot
<img src="https://avatars3.githubusercontent.com/u/56938552?s=100&v=1">

This tutorial is based on the wonderful work of the
[Harvard NLP group](http://nlp.seas.harvard.edu/2018/04/03/attention.html) and [SamLynnEvans](https://github.com/SamLynnEvans/Transformer) 

The goal here is to make new research approachable, silly and fun to the average developer or  interested motivated enthusiast with some basic understanding of linear algebra and programming. We will be building a chatbot and updating it's abilities using the latest research in AI and our own ideas. [Python tutorials](https://www.learnpython.org/) are everywhere on the internet. If you want to learn python so you can learn how Deep Learning Transformers work for Natural Language Processing i suggest learning through [ipython](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) or [jupyter notebooks](https://youtu.be/pxPzuyCOoMI) like this one. For a visual intro to linear algebra i reccomend [Essence of linear algebra](https://youtu.be/fNk_zzaMoSs) and how it applies to neural networks [Deep learning](https://youtu.be/aircAruvnKk) by [3Blue1Brown](https://www.youtube.com/channel/UCYO_jab_esuFRV4b17AJtAw)

Each of these rectangles is a cell, click one of them and Press 
*shift* + *enter/return*, or go to Cell in the nav bar and click "Run Cells" to run each cell, the one below imports `torch` so you can use PyTorch, it also imports some python code that I wrote in the folder *scripts* that I will explain to you after I show you a toy example of how the whole code works together, using a chatbot that says flirtaciously inappropriate things. nltk is the [Natural Language Toolkit](https://www.nltk.org/) that we will be using for things such as synonym matching, that way when you say "lustful", Chloebot knows it means the same thing as "aroused", even if "lustful" is not in Chloebot's vocabulary. To do this nltk will need to download a folder called corpora, right now it is pointing to the *saved* folder, to change this, replace `./saved/` with your prefered path. Run the cells from top to bottom.

In [ ]:
import torch

from scripts.MoveData import *
from scripts.Transformer import *
from scripts.TalkTrain import *

import nltk
nltk.download('wordnet', './saved/') 
from nltk.corpus import wordnet

use files, finder or files explorer to open the csv called `chat_pairs.csv` that is included in the saved folder and add a few of your own conversation pairs to the list. You can see that a comma separates the sentence Chloebot expects to hear, with the sentence Chloebot is expected to respond with, ie

*who are you?,i am chloe*. 

Run the cell below, `data_iter` is an object that gives you training data in the form of batches everytime you call it, `infield` and `outfield` are objects that store the relationship between the strings in Chloe's vocabulary with their indices, for the words Chloe expects to hear and the words Chloe expects to use in response. What do I mean by this? go to Insert and insert a cell below then run `infield.vocab.stoi`, you will see a dictionary of all the words Chloe expects to hear and each word's integer index. The string "smile" might be indexed as `8` and be represented with a vector of length 512. All three represent the same word or *token*. `opt` is a object that stores your preferences such as your learning rate (lr) or path to where you want your neural network weights saved, I will explain all this later.

In [40]:
csv_path = 'saved/chat_pairs.csv'
opt = Options(batchsize = 4)
data_iter, infield, outfield, opt = csv2datatools(csv_path,'en', opt)

OK, now that we have built a data loader, a vocabulary and an object to store our preferences, lets instantiate a Transformer sequence to sequence model. There is alot summoned by this one line, Transformers are the general neural architecture behind many of hyped up / notorious research models of 2017-2019 such as OpenAI's [GPT-2](http://jalammar.github.io/illustrated-gpt2/). 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/OpenAI_Logo.svg/200px-OpenAI_Logo.svg.png"> 

We will take our time with dissecting and understanding it's components later. For now, we have a pre-trained model you can load before fine tuning on the additional conversational pairs you have added. 

In [41]:
emb_dim, n_layers, heads, dropout = 64, 2, 8, 0.1 
opt.save_path = 'saved/weights/model_weights'

model = Transformer(len(infield.vocab), len(outfield.vocab), emb_dim, n_layers, heads, dropout)

if opt.device != -1:
    model = model.cuda()
    
model.load_state_dict(torch.load(opt.save_path))

<All keys matched successfully>

Neural network optimization is a whole field in itself, we will talk more about this in the future. The learning rate `opt.lr` is a hyperparameter whose initial value we choose, it modified the magnitude that the Adam optimizer algorithm will update the weights, aka parameters, of the neural network model during training. As training progresses the learning rate is also changing according to a scheduler [cosine annealing schedule](https://github.com/allenai/allennlp/issues/1642). `epochs` is the number of times we will cycle through the data during training. 

In [42]:
opt.lr = 0.01 # usually 0.01 - 0.0001
opt.epochs = 10 
optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)
scheduler = CosineWithRestarts(optimizer, T_max=num_batches(data_iter))

Now lets training the model on our toy csv dataset, the model should quickly memorize the data. As the loss decreases, the model learns from the data to output the corresponding sequence when fed inputs that are close enough to the training inputs. When the loss is less than 0.1, the responses should start to become coherent. 

In [43]:
model = trainer(model, data_iter, opt, optimizer, scheduler)

0m: epoch 0 loss = 0.039
0m: epoch 1 loss = 0.051
0m: epoch 2 loss = 0.049
0m: epoch 3 loss = 0.062
0m: epoch 4 loss = 0.020
0m: epoch 5 loss = 0.015
0m: epoch 6 loss = 0.008
0m: epoch 7 loss = 0.006
0m: epoch 8 loss = 0.010
0m: epoch 9 loss = 0.006


To talk to the model, your input sentence that Chloe hears, has to be tokenized, aka parsed, converted from strings to a sequence of integers, run through the model, and converted back into strings, we will start our deep dive by dissecting how this is done with Talk.ipynb inside the notebook folder. But before you go, play with the chatbot you just made using the function `talk_to_model()` below. 

In [44]:
sentence = "how are you?" 
opt.k = 2
opt.max_len = 10
sentence = talk_to_model(sentence, model, opt, infield, outfield)
print('Chloe > '+ sentence + '\n')

Chloe > ummm... im embarrased to say



In [ ]:
# when you want to turn off this cell, click Kernel->Interrupt 
while True:
    sentence = input("You > ")
    sentence = talk_to_model(sentence, model, opt, infield, outfield)
    print('Chloe > '+ sentence + '\n')

You > hi
Chloe > why hello there

You > who are you?
Chloe > i am chloe

You > you a robot?
Chloe > yes but you are just a biological robot

